## Search in Rotated Sorted Array (Leetcode 39)

Suppose an array sorted in ascending order is rotated at some pivot unknown to you beforehand.

(i.e., [0,1,2,4,5,6,7] might become [4,5,6,7,0,1,2]).

You are given a target value to search. If found in the array return its index, otherwise return -1.

You may assume no duplicate exists in the array.

Your algorithm's runtime complexity must be in the order of $O(\log{n})$.

```
Example 1:

Input: nums = [4,5,6,7,0,1,2], target = 0
Output: 4
Example 2:

Input: nums = [4,5,6,7,0,1,2], target = 3
Output: -1
```

In [39]:
function search(nums, target) {
    function _search(nums, target, p_dir, p_val) {
        // base cases
        if (nums.length === 0) return -1;
        else if (nums.length === 1) return nums[0] === target ? 0: -1;

        // solution case
        let pivot = Math.floor(nums.length / 2);
        if (nums[pivot] === target) return pivot;

        console.log(p_dir, p_val, nums, pivot, nums[pivot]);
        // first iteration or normal binary case, perform normal binary search
        if (
            (!p_dir && !p_val) ||
            (p_dir === 'right' && p_val > nums[pivot]) ||
            (p_dir === 'left' && p_val < nums[pivot])
        ) {
            if (nums[pivot] < target) {
                let result = _search(nums.slice(pivot + 1), target, 'left', nums[pivot]);
                return result === -1 ? -1 : pivot + 1 + result;
            }
            else
                return _search(nums.slice(0, pivot), target, 'right', nums[pivot]);
        }

        // search left, exclude inner left zone
        else if (nums[pivot] > target && p_dir === 'right' && target > p_val)
            return _search(nums.slice(0, pivot), target, 'right', nums[pivot]);

        // search right, exclude inner right zone
        else if (nums[pivot] < target && p_dir === 'left' && target < p_val) {
            let result = _search(nums.slice(pivot + 1), target, 'left', nums[pivot]);
            return result === -1 ? -1 : pivot + 1 + result;
        }

        // search left, inner left zone first
        else if ((nums[pivot] < target || target < p_val) && p_dir === 'right') {
            let inner_result = _search(nums.slice(pivot + 1), target, 'left', nums[pivot]);
            if (inner_result !== -1) return inner_result
            else return _search(nums.slice(0, pivot), target, 'right', nums[pivot]);
        }

        // search right, inner right zone first
        else if ((nums[pivot] > target || target > p_val) && p_dir === 'left') {
            let inner_result = _search(nums.slice(pivot + 1), target, 'left', nums[pivot]);
            if (inner_result !== -1) return pivot + 1 + inner_result;
            else return _search(nums.slice(0, pivot), target, 'right', nums[pivot]);
        }
    }
    
    return _search(nums, target, null, null);
}

This code, which is wrong but demonstratory, does the following:

* If we binary search right from $a$ to $b$ and $b > a$, perform normal binary search.
* If we binary search right from $a$ to $b$ and $b < a$ and $k > a$ or $k < b$, search the left subspace, then search the right subspace.
* If we binary search right from $a$ to $b$ and $b < a$ and $a < k < b$, perform normal binary search.

This is mirrored on the opposite side.

However this is still too many cases to make for an elegant solution, and this code contains difficult to address compositional problems around the recycling of old unexplored number spaces. Sigh.

The trick to this question isn't to perform modified binary search to look for the target value, but to instead perform modified binary search for look for the *minimum* value, and only *then* perform target binary search, using the pinned values as your guide.

In [7]:
function get_pivot(nums) {
    if (nums.length === 1) return 0
    
    let a_idx = 0;
    let b_idx = nums.length - 1;
    let rightmost_idx_visited = nums.length - 1;
    let idx_min = 0;
    let pivot_idx = null;
    
    while (true) {
        let a = nums[a_idx];
        let b = nums[b_idx];
        
        if ((b > a) && (b_idx > a_idx)) {
            pivot_idx = a_idx;
            break;
        } else if ((b < a) && (b_idx > a_idx)) {  // next_val < prev_val
            [a_idx, b_idx] = [b_idx, a_idx + Math.ceil((b_idx - a_idx) / 2)]
        } else if ((b > a) && (b_idx < a_idx)) {
            if ((a_idx - 1) === b_idx) {
                pivot_idx = a_idx;
                break;
            } else {
                [a_idx, b_idx] = [b_idx, a_idx - 1];
            }
        } else {  // (b < a) && (b_idx < a_idx)
            if (b_idx === 0) {
                pivot_idx = 0;
                break;
            } else {
                [a_idx, b_idx] = [b_idx, b_idx - 1];
            }
        }
    }
    
    return pivot_idx;
}

In [13]:
get_pivot([1])

0

In [15]:
get_pivot([1, 2, 3])

0

In [16]:
get_pivot([1, 2, 3, 0])

3

In [17]:
get_pivot([5, 6, 7, 8, 0, 1, 2, 3, 4])

4

This code implements the first part of that operation: finding the pin. It searches through the space using the following structure:

* On the first iteration, compare the first value with the last value.
* If comparing forward and the right value is smaller than the left value, step to the midpoint of the values, unless the values are neighbors, in which case return the right index.
* If comparing forward and the right value is larger than the left value, stop and return the left value.
* If comparing backwards and the left value is larger than the right value, step back one element in the array, unless the current element is the first element in the array, in which case return 0.

This code appears to be correct, and is $O(\log{n})$ amortized time, and is what I came up with once I had the hint that looking for the midpoint was a good first thing to do. But it has four cases and two subcases, which is still  complex.

Here's a transcription of [a more concise solution](https://leetcode.com/problems/search-in-rotated-sorted-array/discuss/14425/Concise-O(log-N)-Binary-search-solution):

In [30]:
function get_pivot(nums, target) {
    let [lo, hi] = [0, nums.length - 1];
    let mid = null;

    while(lo < hi) {
        mid = (lo + hi) / 2;
        if (nums[mid] > nums[hi]) {
            if (target > nums[mid] || target <= nums[hi]) {
                lo = mid + 1;
            } else {
                hi = mid;
            }
        } else {
            if (target > nums[mid] && target <= nums[hi]) {
                lo = mid + 1;
            } else {
                hi = mid;
            }
        }
    }
    
    if (lo == hi && target != nums[lo]) return -1;
    return lo;
}

OK, the original code:

```
public class Solution {
public int search(int[] nums, int target) {
    int start = 0, end = nums.length - 1;
    while (start < end) {
        int mid = (start + end) / 2;
        if (nums[mid] > nums[end]) {  // eg. 3,4,5,6,1,2
            if (target > nums[mid] || target <= nums[end]) {
                start = mid + 1;
            } else {
                end = mid;
            }
        } else {  // eg. 5,6,1,2,3,4
            if (target > nums[mid] && target <= nums[end]) {
                start = mid + 1;
            } else {
                end = mid;
            }
        }
    }
    if (start == end && target != nums[start]) return -1;
    return start;
}
}
```

This is closer to my original solution. If we know a value is not in the subset, we move the pointer. Why wasn't I capable of writing this originally? Hmm.

...overall, I struggled *immensely* with this problem. =(

## First and Last Index of Element in Sorted Array (Leetcode 34)

Given an array of integers nums sorted in ascending order, find the starting and ending position of a given target value.

Your algorithm's runtime complexity must be in the order of O(log n).

If the target is not found in the array, return [-1, -1].

```
Example 1:

Input: nums = [5,7,7,8,8,10], target = 8
Output: [3,4]
Example 2:

Input: nums = [5,7,7,8,8,10], target = 6
Output: [-1,-1]
```

In [111]:
function leftBinaryBoundSearch(nums, target) {
    if (nums.length === 0) return -1;
    
    let pivot = Math.floor(nums.length / 2);
    if (nums[pivot] === target) {
        let prior = leftBinaryBoundSearch(nums.slice(0, pivot), target);
        return prior === -1 ? pivot : prior;
    } else if (nums[pivot] > target) {
        return leftBinaryBoundSearch(nums.slice(0, pivot), target);
    } else {  // nums[pivot] < target
        let posterior = leftBinaryBoundSearch(nums.slice(pivot + 1), target);
        return (posterior === -1 ? -1 : pivot + 1 + posterior);
    }
}

function rightBinaryBoundSearch(nums, target) {
    if (nums.length === 0) return -1;
    
    let pivot = Math.floor(nums.length / 2);
    if (nums[pivot] === target) {
        let posterior = rightBinaryBoundSearch(nums.slice(pivot + 1), target);
        return posterior === -1 ? pivot : pivot + 1 + posterior;
    } else {  // nums[pivot] > target
        return rightBinaryBoundSearch(nums.slice(0, pivot + 1), target);
    }
    
    let leftwise_pivot = leftBinaryBoundSearch(nums.reverse(), target);
    return leftwise_pivot === -1 ? -1 : nums.length - leftwise_pivot - 1;
}

function binarySearch(nums, target) {
    if (nums.length === 0) return nums;
    
    let pivot = Math.floor(nums.length / 2);
    if (nums[pivot] === target) return pivot;
    else if (nums[pivot] > target) return binarySearch(nums.slice(0, pivot), target);
    else {  // nums[pivot] < target
        let subsearch = binarySearch(nums.slice(pivot + 1), target);
        return subsearch === -1 ? -1 : pivot + 1 + subsearch;
    }
}

function searchRange(nums, target) {
    let pivot = binarySearch(nums, target);
    if (pivot === -1) return [-1, -1];
    
    console.log(nums.slice(pivot));
    let l = leftBinaryBoundSearch(nums.slice(0, pivot + 1), target);
    let r = rightBinaryBoundSearch(nums.slice(pivot), target);
    r = (r === -1 ? r : pivot + r);

    // console.log(l, pivot, r);
    return [l, r];
}

For this first attempt I reasons that I could implement binary search to find the target value easily. At that point the work splits into two cases: finding the left edge with $O(\log{n})$ efficiency, and finding the right edge with $O(\log{n})$ efficiency. I made the really bad mistake of thinking that solving for the right is equivalent to solving for the left on a reversed list, which is completely wrong.

So we need to have another go around. Goddamned I'm bad at this crap.

In [240]:
function binarySearch(nums, target, comparator) {
    if (nums.length === 0) return -1;
    
    let pivot = Math.floor(nums.length / 2);
    if (comparator(nums[pivot]) < 0) {
        // search right subcase
        // console.log(nums, nums[pivot], comparator(nums[pivot]), 'right');
        let posterior = binarySearch(nums.slice(pivot + 1), target, comparator);
        return (posterior === -1 ? posterior : pivot + 1 + posterior);
    } else if (comparator(nums[pivot]) > 0) {
        // search left subcase
        // console.log(nums, nums[pivot], comparator(nums[pivot]), 'left');
        return binarySearch(nums.slice(0, pivot), target, comparator);
    } else {  // === 0
        // console.log(nums, nums[pivot], comparator(nums[pivot]), 'probe right');
        let posterior = binarySearch(nums.slice(pivot + 1), target, comparator);
        return (posterior === -1 ? pivot : pivot + 1 + posterior);
    }
}

function searchRange(nums, target) {
    if (nums.length === 0) return [-1, -1];
    
    let right = binarySearch(nums, target, (v => v - target));
    let left = binarySearch(nums.slice(0, right + 1).reverse(), target, (v => target - v));
    left = (left === -1 ? left : right - left);
    return [left, right];
}

In [241]:
searchRange([], 0)

[ -1, -1 ]

In [242]:
searchRange([0], 0)

[ 0, 0 ]

In [243]:
searchRange([0,0], 0)

[ 0, 1 ]

In [244]:
searchRange([0,0,0], 0)

[ 0, 2 ]

In [245]:
searchRange([0,0,0,0,0], 0)

[ 0, 4 ]

In [246]:
searchRange([0,1], 1)

[ 1, 1 ]

In [247]:
searchRange([1,2], 1)

[ 0, 0 ]

In [248]:
searchRange([1,2], 2)

[ 1, 1 ]

In [249]:
searchRange([0,1,2], 1)

[ 1, 1 ]

In [250]:
searchRange([0,1,1,2], 1)

[ 1, 2 ]

In [251]:
searchRange([0,1,1,1,2], 1)

[ 1, 3 ]

In [252]:
searchRange([0,0,0,1,1,1,2,2,2,3,3,3,4,4,4], 2)

[ 6, 8 ]

In [253]:
searchRange([5,7,7,8,8,10], 6)

[ -1, -1 ]

In [254]:
searchRange([1,2,3], 3)

[ 2, 2 ]

OK, this is a correct solution. This solution solves the problem using two algorithms:

* Given that $a$ is an array in ascending order, find the rightmost entry $k$ in the array such that $k = target$.
* Given that $a$ is an array in descending order, find the rightmost entry $k$ in the array such that $k = target$.

The problem is solved in two steps:

1. Solve for $l = k$ in $a$ (ascending).
2. Solve for $k$ in $\text{reverse}(\text{slice}(0, l))$ (descending). The right boundary $r$ is a well-known function of $k$, e.g. $r = f(k)$.

This solution uses the insight that searching for the rightmost index in an ascending array is equivalent to searching for the rightmost index in a descending array with the comparator reversed.

Getting the boolean logic just right was tricky, as always.

If I had gone to this approach to begin with instead of making an incorrect observation about list reversal this would have been an adequate (and timely enough, given my standards) solve. This solution is elegant, and runs in true $O(\log{n})$ time.

## Valid Sudoku (Leetcode 36)

Determine if a 9x9 Sudoku board is valid. Only the filled cells need to be validated according to the following rules:

* Each row must contain the digits 1-9 without repetition.
* Each column must contain the digits 1-9 without repetition.
* Each of the 9 3x3 sub-boxes of the grid must contain the digits 1-9 without repetition.

A partially filled sudoku which is valid.

The Sudoku board could be partially filled, where empty cells are filled with the character '.'.

```
Example 1:

Input:
[
  ["5","3",".",".","7",".",".",".","."],
  ["6",".",".","1","9","5",".",".","."],
  [".","9","8",".",".",".",".","6","."],
  ["8",".",".",".","6",".",".",".","3"],
  ["4",".",".","8",".","3",".",".","1"],
  ["7",".",".",".","2",".",".",".","6"],
  [".","6",".",".",".",".","2","8","."],
  [".",".",".","4","1","9",".",".","5"],
  [".",".",".",".","8",".",".","7","9"]
]
Output: true
Example 2:

Input:
[
  ["8","3",".",".","7",".",".",".","."],
  ["6",".",".","1","9","5",".",".","."],
  [".","9","8",".",".",".",".","6","."],
  ["8",".",".",".","6",".",".",".","3"],
  ["4",".",".","8",".","3",".",".","1"],
  ["7",".",".",".","2",".",".",".","6"],
  [".","6",".",".",".",".","2","8","."],
  [".",".",".","4","1","9",".",".","5"],
  [".",".",".",".","8",".",".","7","9"]
]
Output: false
Explanation: Same as Example 1, except with the 5 in the top left corner being 
    modified to 8. Since there are two 8's in the top left 3x3 sub-box, it is invalid.
Note:

A Sudoku board (partially filled) could be valid but is not necessarily solvable.
Only the filled cells need to be validated according to the mentioned rules.
The given board contain only digits 1-9 and the character '.'.
The given board size is always 9x9.
```

In [381]:
function fillOptions(board, pos) {
    let row_nums = new Set(board[pos[0]].filter(v => (v !== '.')));
    let col_nums = new Set(board.map(row => row[pos[1]]).filter(v => v !== '.'));

    let x_box_start_pos = Math.floor(pos[0] / 3) * 3;
    let x_box_end_pos = x_box_start_pos + 3;
    let y_box_start_pos = Math.floor(pos[1] / 3) * 3;
    let y_box_end_pos = y_box_start_pos + 3;
    let box = board.slice(x_box_start_pos, x_box_end_pos);
    box = box.map(col => col.slice(y_box_start_pos, y_box_end_pos));
    let box_nums = new Set(box.map(row => row.filter(v => v != '.')).reduce((a, b) => a.concat(b), []));

    let opts = ['1','2','3','4','5','6','7','8','9'];
    return opts.filter(v => (!row_nums.has(v) && !col_nums.has(v) && !box_nums.has(v)));
    return opts;
}

function nextToFill(board) {
    for (let i of Array(board.length).keys()) {
        for (let j of Array(board.length).keys()) {
            // console.log(i, j);
            if (board[i][j] === '.') return [i,j];
        }
    }
    return null;
}

function isValidSudoku(board) {
    let pos = nextToFill(board);
    if (!pos) return true;
    
    for (let opt of fillOptions(board, pos)) {
        board[pos[0]][pos[1]] = opt;
        if (isValidSudoku(board)) {
            return true;
        } else {
            board[pos[0]][pos[1]] = '.'
        }
    }
    
    return false;
}

In [304]:
nextToFill([['.']])

[ 0, 0 ]

In [305]:
nextToFill([['1']])

null

In [306]:
nextToFill([['.', '.'],
            ['.', '.']])

[ 0, 0 ]

In [308]:
nextToFill([['1', '.'],
            ['.', '.']])

[ 0, 1 ]

In [310]:
nextToFill([['1', '2'],
            ['.', '.']])

[ 1, 0 ]

In [312]:
nextToFill([['1', '2'],
            ['3', '.']])

[ 1, 1 ]

In [313]:
nextToFill([['1', '2'],
            ['3', '4']])

null

In [332]:
fillOptions([['.']], [0, 0])

[ '1', '2', '3', '4', '5', '6', '7', '8', '9' ]

In [335]:
fillOptions([['.','1']], [0, 0])

[ '2', '3', '4', '5', '6', '7', '8', '9' ]

In [336]:
fillOptions([['.','2', '3', '4', '5', '6', '7', '8', '9']], [0, 0])

[ '1' ]

In [337]:
fillOptions([['.','2', '3'],
             ['4', '5', '6'],
             ['7', '8', '9']], [0, 0])

[ '1' ]

In [342]:
fillOptions([['.'],
             ['2'],
             ['3'],
             ['4'],
             ['5'],
             ['6'],
             ['7'],
             ['8'],
             ['9']], [0, 0])

[ [], [ '2' ], [ '3' ] ]


[ '1' ]

In [353]:
fillOptions([['1', '2', '3', '.','5', '6'],
             ['7', '8', '9', '1', '2', '3'],
             ['4', '5', '6', '7', '8', '9']], [0, 3])

[ '4' ]

In [359]:
isValidSudoku([['.']])

true

In [360]:
isValidSudoku([['1', '2', '3', '4', '5', '6', '7', '8', '.']])

true

In [382]:
isValidSudoku([
  ["8","3",".",".","7",".",".",".","."],
  ["6",".",".","1","9","5",".",".","."],
  [".","9","8",".",".",".",".","6","."],
  ["8",".",".",".","6",".",".",".","3"],
  ["4",".",".","8",".","3",".",".","1"],
  ["7",".",".",".","2",".",".",".","6"],
  [".","6",".",".",".",".","2","8","."],
  [".",".",".","4","1","9",".",".","5"],
  [".",".",".",".","8",".",".","7","9"]
])

false

In [383]:
isValidSudoku([
  ["5","3",".",".","7",".",".",".","."],
  ["6",".",".","1","9","5",".",".","."],
  [".","9","8",".",".",".",".","6","."],
  ["8",".",".",".","6",".",".",".","3"],
  ["4",".",".","8",".","3",".",".","1"],
  ["7",".",".",".","2",".",".",".","6"],
  [".","6",".",".",".",".","2","8","."],
  [".",".",".","4","1","9",".",".","5"],
  [".",".",".",".","8",".",".","7","9"]
])

true

In [384]:
isValidSudoku([
    [".","8","7","6","5","4","3","2","1"],
    ["2",".",".",".",".",".",".",".","."],
    ["3",".",".",".",".",".",".",".","."],
    ["4",".",".",".",".",".",".",".","."],
    ["5",".",".",".",".",".",".",".","."],
    ["6",".",".",".",".",".",".",".","."],
    ["7",".",".",".",".",".",".",".","."],
    ["8",".",".",".",".",".",".",".","."],
    ["9",".",".",".",".",".",".",".","."]])

false

This is a classic recursion problem. There's some dynamic programming tricks that can be applied to it. The question actually asks for the mirror of the problem: to solve for the filled positions. Which is significantly easier.

## Combination Sum (Leetcode 39)

In [474]:
function combineSets(a, b) {
    let out = [];
    for (let _a of a) {
        for (let _b of b) {
            out.push(_a.concat(_b));
        }
    }
    return out;
}

function permutationSum(candidates, target) {
    // candidates are already in sorted order
    if (candidates.length === 0) return [];
    
    let memo = [];
    let c_idx = 0;
    for (let i of Array(target + 1).keys()) {
        memo[i] = [];
        // console.log(memo);
        while ((c_idx < candidates.length) && (candidates[c_idx] === i)) {
            memo[i].push([candidates[c_idx]])
            // console.log(memo);
            c_idx += 1;
        }
        for (let j of Array(Math.floor(i / 2) + 1).keys()) {
            let k = i - j;
            memo[i] = memo[i].concat(combineSets(memo[j], memo[k]));
        }
    }
    
    return memo[target];
}

In [475]:
permutationSum([1,2,3], 1)

[ [ 1 ] ]

In [476]:
permutationSum([1,2,3], 2)

[ [ 2 ], [ 1, 1 ] ]

In [477]:
permutationSum([1,2,3], 3)

[ [ 3 ], [ 1, 2 ], [ 1, 1, 1 ] ]

In [478]:
permutationSum([1, 2, 3, 4, 5, 6], 6)

[ [ 6 ],
  [ 1, 5 ],
  [ 1, 1, 4 ],
  [ 1, 1, 1, 3 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ],
  [ 1, 1, 2, 2 ],
  [ 1, 1, 2, 1, 1 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ],
  [ 1, 2, 3 ],
  [ 1, 2, 1, 2 ],
  [ 1, 2, 1, 1, 1 ],
  [ 1, 1, 1, 3 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ],
  [ 2, 4 ],
  [ 2, 1, 3 ],
  [ 2, 1, 1, 2 ],
  [ 2, 1, 1, 1, 1 ],
  [ 2, 2, 2 ],
  [ 2, 2, 1, 1 ],
  [ 2, 1, 1, 2 ],
  [ 2, 1, 1, 1, 1 ],
  [ 1, 1, 4 ],
  [ 1, 1, 1, 3 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ],
  [ 1, 1, 2, 2 ],
  [ 1, 1, 2, 1, 1 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ],
  [ 3, 3 ],
  [ 3, 1, 2 ],
  [ 3, 1, 1, 1 ],
  [ 1, 2, 3 ],
  [ 1, 2, 1, 2 ],
  [ 1, 2, 1, 1, 1 ],
  [ 1, 1, 1, 3 ],
  [ 1, 1, 1, 1, 2 ],
  [ 1, 1, 1, 1, 1, 1 ] ]

In [494]:
function combineUniqueSets(a, b, uniquifier) {
    let out = [];
    for (let _a of a) {
        for (let _b of b) {
            let proposal = _a.concat(_b);
            let proposal_str = proposal.sort((a, b) => a - b).join('');
            if (!uniquifier.has(proposal_str)) {
                out.push(_a.concat(_b));
                uniquifier.add(proposal_str);
            }
        }
    }
    return out;
}

function combinationSum(candidates, target) {
    if (candidates.length === 0) return [];
    candidates = candidates.sort((a, b) => a - b);
    
    let memo = [];
    let uniquifier = new Set();
    let c_idx = 0;
    for (let i of Array(target + 1).keys()) {
        memo[i] = [];
        while ((c_idx < candidates.length) && (candidates[c_idx] === i)) {
            memo[i].push([candidates[c_idx]])
            uniquifier.add(candidates[c_idx].toString());
            c_idx += 1;
        }
        for (let j of Array(Math.floor(i / 2) + 1).keys()) {
            let k = i - j;
            memo[i] = memo[i].concat(combineUniqueSets(memo[j], memo[k], uniquifier));
        }
    }
    
    return memo[target];
}

The difference between finding unique permutations of things and finding unique combinations of things is linear work and additiion space. For a combination with replacement problem such as this one, the easiest solution (IMO) is usually to solve for permutations with replacement, then inline uniqueness checks. This is what I've done here.

This ia dynamic programming solution (memoization, nothing fancy). Combining items from the memo is constant work. We do this $n /2$ times per iteration, and we iterate $n$ times, making this algorithm $O(n^2)$.

## Combination Sum II (Leetcode 40)

This time without repeats.

## Multiply Strings (Leetcode 43)

Given two non-negative integers num1 and num2 represented as strings, return the product of num1 and num2, also represented as a string.

```
Example 1:

Input: num1 = "2", num2 = "3"
Output: "6"
Example 2:

Input: num1 = "123", num2 = "456"
Output: "56088"
Note:

The length of both num1 and num2 is < 110.
Both num1 and num2 contain only digits 0-9.
Both num1 and num2 do not contain any leading zero, except the number 0 itself.
You must not use any built-in BigInteger library or convert the inputs to integer directly.
```

In [501]:
[...Array(2 + 2 - 1).keys()].map(v => v + 1)

[ 1, 2, 3 ]

In [538]:
function multiply(num1, num2) {
    let out = Array(num1.length + num2.length - 1);
    
    function place_digit_combos(num1, num2) {
        for (let j of Array(num1.length).keys()) {
            for (let k of Array(num2.length).keys()) {
                let zeros = j + k;
                if (out[zeros]) {
                    out[zeros] = out[zeros].concat([[num1[j], num2[k]]]);
                } else {
                    out[zeros] = [[num1[j], num2[k]]]
                }
            }
        }
        return out;
    }
    
    let digits_sorted = place_digit_combos(num1, num2);
    let digits_sorted_idx = 0;
    while (digits_sorted_idx < digits_sorted.length) {
        let subsum = 0;
        for (let [digit1, digit2] of digits_sorted_idx) {
            subsum = parseInt(digit1) * parseInt(digit2);
            // more code...more tedious than difficult
        }
    }
}


In [539]:
multiply('12', '12')

[ [ [ '1', '1' ] ],
  [ [ '1', '2' ], [ '2', '1' ] ],
  [ [ '2', '2' ] ] ]

## Permutations (Leetcode 46)

In [561]:
function permute(nums) {
    if (nums.length <= 1) return [nums];
    
    let out = [];
    for (let i of Array(nums.length).keys()) {
        let subperms = permute(nums.slice(0, i).concat(nums.slice(i + 1)));
        for (let subperm of subperms) {
            // console.log(subperm);
            out.push([nums[i], ...subperm]);
        }
    }
    return out;
}

In [562]:
permute([1,2,3])

[ [ 1, 2, 3 ],
  [ 1, 3, 2 ],
  [ 2, 1, 3 ],
  [ 2, 3, 1 ],
  [ 3, 1, 2 ],
  [ 3, 2, 1 ] ]

Straightforward recursion (backtracking). Solution is $O(n!)$.

## Permutations II (Leetcode 47)

Given a collection of numbers that might contain duplicates, return all possible unique permutations.

```
Example:

Input: [1,1,2]
Output:
[
  [1,1,2],
  [1,2,1],
  [2,1,1]
]
```

This question was a learning experience.

It's easy to modify the code from the previous section to solve this problem as well by adding deduplication to the routine. This is a linear cost, easy to program, and doesn't affect the $O$ performance of the algorithm.

I thought how to invert this problem and solve it using dynamic programming. The set of solutions for $(a, b, c)$ relies on the sets of solutions $\{(a - 1, b, c), (a, b - 1, c), (a, b, c - 1)\}$. However, this is a recursive backwards-facing solution space, and I couldn't puzzle out how to iterate through the cases without implementing a whole second complicated algorithm.

Ultimately the solution was to add a stop condition to the insertion operation in the original algorithm. I tested this out on paper and it works. I don't know how I would discover this stop condition on my own, in my own analysis, but now that I know about it, this is a useful trick that will come in handy in the future.

```python
def permuteUnique(self, nums):
    ans = [[]]
    for n in nums:
        new_ans = []
        for l in ans:
            for i in range(len(l)+1):
                new_ans.append(l[:i]+[n]+l[i:])
                if i<len(l) and l[i]==n: break              #handles duplication
        ans = new_ans
    return ans
```

## Rotate Image (Leetcode 48)

In [596]:
function rotate(matrix) {
    for (let x of Array(matrix.length).keys()) {
        for (let y of Array(matrix.length - x - 1).keys()) {
            let [x_c, y_c] = [matrix.length - 1 - y, matrix.length - 1 - x];
            [matrix[x][y], matrix[x_c][y_c]] = [matrix[x_c][y_c], matrix[x][y]];
        }
    }
    
    for (let x of Array(Math.floor(matrix.length / 2)).keys()) {
        for (let y of Array(matrix.length).keys()) {
            let [x_c, y_c] = [matrix.length - 1 - x, y];
            [matrix[x][y], matrix[x_c][y_c]] = [matrix[x_c][y_c], matrix[x][y]];
        }
    }
}

In [597]:
rotate([['1', '2'], ['3', '4']])

[ [ '3', '1' ], [ '4', '2' ] ]

## Group Anagrams (Leetcode 49)

In [16]:
function groupAnagrams(strs) {
    let pos_map = new Map;
    let out = [];
    
    for (let str of strs) {
        let char_seq = [...str].sort((a, b) => a.charCodeAt(0) - b.charCodeAt(0)).join('');
        
        if (pos_map.has(char_seq)) {
            out[pos_map.get(char_seq)].push(str);
        } else {
            pos_map.set(char_seq, out.length);
            out[out.length] = [str];
        }
    }
    return out;
}

In [22]:
groupAnagrams(['abc', 'acb', 'cab', 'cba'])

[ [ 'abc', 'acb', 'cab', 'cba' ] ]

Two words being anagrams of one another reduces to those two words having a unique sorted sequences of characters, e.g. `eat` and `tea` would both reduce to `aet`. We map each index in the output array to a specific output sequence in iteration discovery order, using a hash map to make find-index into an $O(1)$ operation. Generating the sorted character sequence is an $O(m\log{m})$ operation, where $m$ is the length of the word. So the overall algorithm is $O(nm\log{m})$.

Compare with this following Python solution:

```python
def groupAnagrams(self, strs):
    d = {}
    for w in sorted(strs):
        key = tuple(sorted(w))
        d[key] = d.get(key, []) + [w]
    return d.values()
```

This algorithm is $O(n\log{n})$. Which of these two is better seems like it'd depend mostly on the length of the words.

## House Robbers (Leetcode 198)

Apparently a pretty famous problem, discussed it on the roof last night.

In [53]:
function rob(nums) {
    if (!nums) return 0;
    else if (nums.length === 1) return nums[0];
    else if (nums.length === 2) return nums[0] > nums[1] ? nums[0] : nums[1];
    else {
        let best_submax = 0;
        for (let numidx of Array(nums.length).keys()) {
            let left_r_bound = numidx === 0 ? 0 : numidx - 1;
            let left = rob(nums.slice(0, left_r_bound));
            let right = rob(nums.slice(numidx + 2));
            let tot = nums[numidx] + left + right;
            if (tot > best_submax) best_submax = tot;
        }
        return best_submax;
    }
}

In [54]:
rob([])

0

In [55]:
rob([1])

1

In [56]:
rob([1,2])

2

In [57]:
rob([1,2,3])

4

In [58]:
rob([1,2,3,1])

4

In [59]:
rob([2,7,9,3,1])

12

<s>This solution is $O(n!)$ because each case for an array of length $n$ breaks down (roughly) into $n$ cases of arrays of length $n - 2$. That requires a recursion tree of cost $(1)(3)(5)\ldots(n - 2)(n) \leq n!$</s>

The actual answer for time complexity: $O(2^n)$. Why you get this lower bound:

$$W(n=7) = W(5) + W(4) + (W(1) + W(3)) + W(2) + W(2) + (W(1) + W(3)) + W(4) + W(5)$$
$$= 2(W(5) + W(4) + 2(W(3) + W(1))$$
$$= 2(W(n-2)) + 2(W(n-3)) + 4(W(n-4)) + 4(W(n-5)) + \ldots \: | \: n > 0$$
$$= \sum_{k=0}^{k=\lfloor n / 2 \rfloor} 2^k (W(n- 2 - 2k) + W(n- 3 - 2k))$$
$$\leq 2^n$$

"[Because Fibbonacci](https://medium.com/outco/how-to-solve-the-house-robber-problem-f3535ebaef1b)".

This solution is correct but extremely slow. The Leetcode exercise asks for an answer for the following, the computation time for which exceeds the time limit:

```
rob([183,219,57,193,94,233,202,154,65,240,97,234,100,249,186,66,90,238,168,
     128,177,235,50,81,185,165,217,207,88,80,112,78,135,62,228,247,211])
```

There are two ways to get speed-ups. The most immediately implementable way is to perform memoization. The other, much more difficult way is to reverse the problem into one in dynamic programming.

In [81]:
function rob(nums) {
    function _rob(nums, memo) {
        // console.log(nums, memo);
        
        if (!nums) return [0, memo];
        else if (nums.length === 1) return [nums[0], memo];
        else if (nums.length === 2) return nums[0] > nums[1] ? [nums[0], memo] : [nums[1], memo];
        else if (memo.has(nums.join(','))) return [memo.get(nums.join(',')), memo];
        else {
            let best_submax = 0;
            for (let numidx of Array(nums.length).keys()) {
                let left_r_bound = numidx === 0 ? 0 : numidx - 1;
                let left = nums.slice(0, left_r_bound);
                let left_memo_key = left.join(',');
                if (memo.has(left_memo_key)) {
                    left = memo.get(left_memo_key);
                } else {
                    [left, memo] = _rob(nums.slice(0, left_r_bound), memo);
                    memo.set(left_memo_key, left);
                }
                
                let right = nums.slice(numidx + 2);
                let right_memo_key = right.join(',');
                if (memo.has(right_memo_key)) {
                    right = memo.get(right_memo_key);
                } else {
                    [right, memo] = _rob(nums.slice(numidx + 2), memo);
                    memo.set(right_memo_key, right);
                }
                
                let tot = nums[numidx] + left + right;
                if (tot > best_submax) best_submax = tot;
            }
            memo.set(nums.join(','), best_submax);
            return [best_submax, memo];
        }
    }
    
    let memo = new Map();    
    return _rob(nums, memo)[0];
}

In [83]:
rob([])

0

In [84]:
rob([1])

1

In [85]:
rob([2])

2

In [86]:
rob([2,7,9,3,1])

12

In [87]:
rob([183,219,57,193,94,233,202,154,65,240,97,234,100,249,186,66,90,238,168,
     128,177,235,50,81,185,165,217,207,88,80,112,78,135,62,228,247,211])

3365

Memoization did the trick. How fast is this solution? Well, it guaranteed that we only ever solve each possible subproblem once, and we are guaranteed to visit every single subproblem. So the answer is the number of neighborhoods of every possible size in the ordered list. There are $n$ one-element neighborhoods, $n - 1$ two-element neighborhoods, as so on. So memoization turns this solution into a $n + (n - 1) + \ldots + 1 = \sum_{i=1}^n i = \frac{n(n + 1)}{2} \leq n^2 \implies O(n^2)$ solution.

At first I thought that dymanic programming would not speed up the $O$ speed of the solution because it would only eliminate redudant memo checks. But it turns out that there is a deeper recurrance relationship within the call stack that dynamic programming can indeed eliminate. We use the relation that $f(arr) \leq f(arr + [n])$; e.g. that the solution to the problem when one additional house is added is equivalent to the solution the previous subproblem, plus potentially an ascending pattern of values:

![](https://miro.medium.com/max/5462/1*CQqAenqZdRXQljynJ8s2wQ.png)

This solution is $O(n)$. =(

Things that didn't go right here:
* Finding the time complexity boundary of the naive recursive solution.
* One of the explainers makes the following comment about the memoized solution:

  > Much better, this should run in O(n) time. 
  
  Which I don't believe.
* Not finding the deeper subproblem division.

Overall, this problem definitely doesn't qualify as "Easy".